In [1]:
# Setup the SparkSession
from anovos.shared.spark import spark

# Import function for data loading
from anovos.data_ingest.data_ingest import read_dataset
from anovos.data_analyzer.stats_generator import measures_of_percentiles

# Import other modules
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window

2022-03-22 16:45:49.231 | INFO     | anovos.shared.spark:init_spark:54 - Getting spark session, context and sql context app_name: Anovos_pipeline
Ivy Default Cache set to: /Users/sinuochen/.ivy2/cache
The jars for the packages stored in: /Users/sinuochen/.ivy2/jars
:: loading settings :: url = jar:file:/Users/sinuochen/server/spark-2.4.8-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.github.histogrammar#histogrammar_2.11 added as a dependency
io.github.histogrammar#histogrammar-sparksql_2.11 added as a dependency
org.apache.spark#spark-avro_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2fd7a06b-842f-404d-b11a-891f95e93706;1.0
	confs: [default]
	found io.github.histogrammar#histogrammar_2.11;1.0.20 in central
	found io.github.histogrammar#histogrammar-sparksql_2.11;1.0.20 in central
	found org.apache.spark#spark-avro_2.11;2.4.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution

In [2]:
output_path = "../data/pre_saved_stats/"

## Read Data

In [3]:
df = read_dataset(
    spark,
    file_path='../data/application_train_modified.csv',
    file_type='csv',
    file_configs={'header': 'True', 'delimiter': ',', 'inferSchema': 'True'})

In [4]:
stability_cols = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                  'DAYS_EMPLOYED', 'DAYS_BIRTH', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN',
                  'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
cols_sample = ['SK_ID_CURR', 'TARGET', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL', 
               'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 
               'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 
               'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
               'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'CODE_GENDER', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 
               'DAYS_ID_PUBLISH', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 
               'NAME_INCOME_TYPE', 'NAME_TYPE_SUITE', 'ORGANIZATION_TYPE', 'REGION_RATING_CLIENT_W_CITY', 
               'LIVE_CITY_NOT_WORK_CITY']

## Generate Historical Datasets for Data Stability

In [5]:
df = df.select(stability_cols+cols_sample).withColumn("id", F.monotonically_increasing_id())\
    .withColumn("id", F.row_number().over(Window.orderBy("id")))

In [6]:
from anovos.data_analyzer.stats_generator import measures_of_percentiles
odf = measures_of_percentiles(spark, idf = df)
odf.toPandas()

,attribute,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
0,COMMONAREA_MEDI,0.0000,0.0000,0.0000,0.0007,0.0079,0.0208,0.0513,1.053000e-01,1.607000e-01,3.769000e-01,1.000000e+00
1,CNT_CHILDREN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.000000e+00,2.000000e+00,3.000000e+00,1.900000e+01
2,CNT_FAM_MEMBERS,1.0000,1.0000,1.0000,1.0000,2.0000,2.0000,3.0000,3.000000e+00,4.000000e+00,5.000000e+00,2.000000e+01
3,EXT_SOURCE_3,0.0005,0.0647,0.1547,0.2276,0.3706,0.5353,0.6691,7.490000e-01,7.863000e-01,8.328000e-01,8.960000e-01
4,REGION_RATING_CLIENT_W_CITY,1.0000,1.0000,1.0000,1.0000,2.0000,2.0000,2.0000,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00
5,FLAG_CONT_MOBILE,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
6,DAYS_BIRTH,-54425.0000,-24420.0000,-23203.0000,-22182.0000,-19681.0000,-15750.0000,-12414.0000,-1.028500e+04,-9.406000e+03,-8.263000e+03,-7.489000e+03
7,DAYS_ID_PUBLISH,-7197.0000,-5445.0000,-4943.0000,-4723.0000,-4299.0000,-3254.0000,-1720.0000,-7.300000e+02,-3.750000e+02,-6.100000e+01,0.000000e+00
8,AMT_GOODS_PRICE,40500.0000,67500.0000,135000.0000,180000.0000,238500.0000,450000.0000,679500.0000,1.093500e+06,1.305000e+06,1.800000e+06,4.050000e+06
9,BASEMENTAREA_MEDI,0.0000,0.0000,0.0000,0.0000,0.0437,0.0758,0.1116,1.678000e-01,2.230000e-01,3.835000e-01,1.000000e+00


In [7]:
df1 = df.where(F.col('id')<=500).drop('id')
df2 = df.where((F.col('id')>2000) & (F.col('id')<=2500)).drop('id')
df3 = df.where((F.col('id')>3000) & (F.col('id')<=3500)).drop('id')
df4 = df.where((F.col('id')>4000) & (F.col('id')<=4500)).drop('id')
df5 = df.where((F.col('id')>5000) & (F.col('id')<=5500)).drop('id')
df6 = df.where((F.col('id')>6000) & (F.col('id')<=6500)).drop('id')

## Calculate Stability Index
The following code generate a dataset in **data/pre_saved_stats/stability**

In [8]:
from anovos.drift.detector import stability_index_computation
df_stability = stability_index_computation(spark, df1, df2, df3, df4, df5, df6, 
                                           appended_metric_path=output_path+'stability', threshold=2)
df_stability.toPandas()

,attribute,mean_cv,stddev_cv,kurtosis_cv,mean_si,stddev_si,kurtosis_si,stability_index,flagged
0,AMT_INCOME_TOTAL,0.0275,0.1290,0.7320,4,2,0.0,2.6,0
1,EXT_SOURCE_3,0.0210,0.0507,0.0837,4,3,3.0,3.5,0
2,AMT_REQ_CREDIT_BUREAU_DAY,0.6171,0.5455,NaN,0,0,NaN,NaN,1
3,AMT_GOODS_PRICE,0.0338,0.0607,0.1337,3,3,2.0,2.8,0
4,AMT_CREDIT,0.0321,0.0542,0.1249,3,3,2.0,2.8,0
5,DAYS_EMPLOYED,0.0784,0.0291,0.1029,3,4,2.0,3.1,0
6,EXT_SOURCE_1,0.0199,0.0244,0.0424,4,4,3.0,3.8,0
7,SK_ID_CURR,0.0221,0.0203,0.0055,4,4,4.0,4.0,0
8,FLAG_CONT_MOBILE,0.0019,0.5197,NaN,4,0,NaN,NaN,1
9,EXT_SOURCE_2,0.0157,0.0337,0.0621,4,3,3.0,3.5,0


In [9]:
df_intermediate = read_dataset(spark, output_path+"stability", "csv", {'header':True,'inferSchema':True})
df_intermediate.show(10, False)

+---+-------------------------+-------------------+-------------------+------------------+
|idx|attribute                |mean               |stddev             |kurtosis          |
+---+-------------------------+-------------------+-------------------+------------------+
|1  |AMT_INCOME_TOTAL         |171603.68931000002 |96555.97627397152  |8.52189894751671  |
|1  |EXT_SOURCE_3             |0.5081177091454547 |0.20482427010919332|2.210134393852552 |
|1  |AMT_REQ_CREDIT_BUREAU_DAY|0.01182033096926714|0.1610164569107727 |288.4874748720538 |
|1  |AMT_GOODS_PRICE          |535599.0           |373265.5529848559  |5.6828728646377025|
|1  |AMT_CREDIT               |595513.566         |401230.1596290132  |4.62314261396233  |
|1  |DAYS_EMPLOYED            |57880.788          |136288.82416576945 |4.291830032742117 |
|1  |EXT_SOURCE_1             |0.5065364652985783 |0.2108052944623017 |2.147079910705088 |
|1  |SK_ID_CURR               |100288.692         |165.87126239161887 |1.809212006729172 |

## Save Statistics for Full Report Generation

In [10]:
from anovos.data_ingest.data_ingest import select_column, write_dataset
report_stats = output_path + 'report_stats/'
df_sample = select_column(df.sample(False, 0.033, 13), list_of_cols=cols_sample)

The following code generate required data in **data/pre_saved_stats/report_stats** which will be used to generate the full report in use case demo.

In [12]:
# Save required statistics for drift detection
from anovos.drift.detector import statistics, stability_index_computation
import pyspark.sql.functions as F

source_df = select_column(df.where(F.col('TARGET')==1).sample(False, 0.08, 7), list_of_cols=cols_sample)
odf1 = statistics(spark, df_sample, source_df, drop_cols='SK_ID_CURR', method_type="all", source_path=report_stats)
write_dataset(odf1, report_stats+'drift_output/', 'parquet',{'mode':'overwrite'})

#Save required statistics for stability analysis
df7 = select_column(df.sample(False, 0.0065, 7), list_of_cols=cols_sample)
odf2 = stability_index_computation(spark, df7, drop_cols='SK_ID_CURR',
                                   existing_metric_path=output_path+'stability', 
                                   appended_metric_path=report_stats+'stabilityIndex_metrics',
                                   threshold=2)
write_dataset(odf2, report_stats+'stability_index', 'parquet',{'mode':'overwrite'})